In [ ]:
import csv

def find_index(l, v):
  for i in range(len(l)):
    if l[i][1] == v:
      return i

def find_off_between(front_list, tx1, tx2):
  for i in range(len(front_list)):
    if front_list[i][1] == tx1:
      for j in range(i+1, len(front_list)):
        if front_list[j][0] == 0:
          return True
        if front_list[j][1] == tx2:
          return False

def average_of_part(inter, start, end):
  sum = 0
  counter = 0
  for i in range(start, end):
    sum=sum+inter[i]
    counter=counter+1
  return sum/counter

with open("front", "r") as f:
    reader = csv.reader(f)
    front_data = list(reader)


with open("rear", "r") as f:
    reader = csv.reader(f)
    rear_data = list(reader)

front_list = []
rear_list = []
for row in front_data:
        front_list.append((int(row[0]), int(row[1])))

for row in rear_data:
        rear_list.append((int(row[0]), int(row[1])))

#print(rear_data)
#PKT loss rate

send_count = 0
first_tx_ind = find_index(front_list, rear_list[0][0])
last_tx_ind = find_index(front_list, rear_list[-1][0])
#print(first_tx_ind, ", ", rear_list[0][0], ", ", last_tx_ind, ", ", rear_list[-1][0])
for i in range(first_tx_ind, last_tx_ind):
  if front_list[i][0] == 1:
    send_count = send_count + 1

loss_rate = len(rear_list)/send_count
print("Packet delivery rate: ", loss_rate)


diff = []
for i in range(len(rear_list)):
    diff.append(rear_list[i][1] - rear_list[i][0])
print("")
print("Latency statistics:")
print("Min latency TX timestamp:", rear_list[diff.index(min(diff))][0])
diff.sort()
print("Min：", diff[0])
print("Max：", diff[-1])
print("Average：", sum(diff)/len(diff))
print("<90%：", diff[int((len(diff)*0.9)+0.5)])
print("Total counts:", len(diff))

count_pair = []
inter = []
i = 0
for i in range(len(rear_list)):
  if i+1 >= len(rear_list):
    break
  tx1 = rear_list[i][0]
  tx2 = rear_list[i+1][0]
  if find_off_between(front_list, tx1, tx2) != True:
    count_pair.append([tx1, tx2])
    inter.append(tx2-tx1)

#for i in range(len(count_pair)):
#  if inter[i] > 200:
#    print(count_pair[i], ": ", inter[i])
print("")
print("FPS statistics: ")
print("Max interval: ", max(inter), "(TX1, TX2): ", count_pair[inter.index(max(inter))])
inter.sort()
less_than_90_perc_inter = average_of_part(inter, 0, int((len(inter)*0.9)+0.5))
less_than_90_perc_fps = 1000/(sum(inter[:int((len(inter)*0.9)+0.5)])/len(inter[:int((len(inter)*0.9)+0.5)]))
less_than_95_perc_inter = average_of_part(inter, 0, int((len(inter)*0.95)+0.5))
less_than_95_perc_fps = 1000/(sum(inter[:int((len(inter)*0.95)+0.5)])/len(inter[:int((len(inter)*0.9)+0.5)]))
print("Average interval:", sum(inter)/len(inter))
print("FPS: ", 1000/(sum(inter)/len(inter)))
print("<90% interval: ", less_than_90_perc_inter)
print("<90% FPS: ", less_than_90_perc_fps)
print("<95% interval: ", less_than_95_perc_inter)
print("<95% FPS: ", less_than_95_perc_fps)
